
#Applying BRFSS Weights Example

This notebook demonstrates how to convert BRFSS data from their integers to strings and apply weights to calculate weighted percentages from a demographic column. We've already injected the BRFSS data into a [BigQuery table](https://console.cloud.google.com/bigquery?ws=!1m5!1m4!4m3!1srci-flotus!2sCDC!3sbrfss_2022) through the '+ Add' button via a DataPrep job. We downloaded the data from the CDC BRFSS website. It required a little bit of cleanup - we had to modify a couple of column names to remove some special characters.
The end results gets written to a [BigQuery table](https://console.cloud.google.com/bigquery?ws=!1m5!1m4!4m3!1srci-flotus!2sCDC!3sweighted_demo).




Install required packages

In [ ]:
!pip install google-cloud-bigquery pydata-google-auth

Import necessary libraries



In [ ]:
import pandas as pd
from google.cloud import bigquery

Initiate BigQuery client to run queries with

In [ ]:
client = bigquery.Client()

Set your variables for BigQuery table to use


In [ ]:
PROJECT_ID = 'rci-flotus'
TABLE_ID = 'CDC.brfss_2022'

Define query

In [ ]:
query = f"""
SELECT
  id,
  ageg5yr,
  llcpwt
FROM
  `{PROJECT_ID}.{TABLE_ID}`
"""


Create a map to convert BRFSS ints to their values. You can find these values (including ranges of integers) using the [brfss codebook found on the CDC website](https://www.cdc.gov/brfss/annual_data/annual_2022.html).

In [ ]:
int_values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
int_to_string_map = {
    1: '18 - 24',
    2: '25 - 29',
    3: '30 - 34',
    4: '35 - 39',
    5: '40 - 44',
    6: '45 - 49',
    7: '50 - 54',
    8: '55 - 59',
    9: '60 - 64',
    10: '65 - 69',
    11: '70 - 74',
    12: '75 - 79',
    13: '80+',
    14: 'N/A'
}

Define our functions for calculating weighted percentages and mapping the values using our maps above.



In [ ]:
def weighted_percentage(data, numeric_column, weight_column):
    valid_data = data.dropna(subset=[numeric_column])

    # Calculate the weighted percentage as a Series. Normally we'd multiply
    # the value of each row by the weight in the divisor, however, since
    # we're looking at demographic values, each value is essentially 1 (i.e.)
    # I'm between 18-24 and that only gets counted once.
    weighted_percent = (valid_data[weight_column]
                        .groupby(valid_data[numeric_column]).sum()
                        / valid_data[weight_column].sum() * 100
                       ).sort_values(ascending=False)

    # Convert the Series to a DataFrame and rename the column
    weighted_percentage_df = weighted_percent.reset_index()

    # Rename the column to reflect the weighted percent calculation
    weighted_percentage_df.columns = [numeric_column, 'weighted_percentage']

    return weighted_percentage_df

# Function to map integers to strings using the dictionary
def map_integers_to_strings(int_list, mapping_dict):
    return [mapping_dict.get(i, 'Unknown') for i in int_list]

Create a Pandas dataframe using the query



In [ ]:
df = client.query(query).to_dataframe()
# print number of rows in dataframe
print(df.index.size)

50


Calculate the frequency and percentage of each row, unweighted. We'll also map the description values from the integer values for a more informative result. We'll choose "AGEG5YR" - a column whose value corresponds to an age range (e.g. 30 - 34).

In [ ]:
# Map the description values with the integer values
df['ageg5yr_mapped'] = df['ageg5yr'].map(int_to_string_map)

# Frequency calculation
frequency = df['ageg5yr_mapped'].value_counts().reset_index()
frequency.columns = ['value_label', 'frequency']

# Total count for percentage calculation
total_count = len(df)
frequency['percentage'] = (frequency['frequency'] / total_count) * 100

print(frequency)

   value_label  frequency  percentage
0      60 - 64          6        12.0
1      40 - 44          5        10.0
2      65 - 69          5        10.0
3      45 - 49          5        10.0
4      55 - 59          5        10.0
5      18 - 24          5        10.0
6      30 - 34          4         8.0
7      70 - 74          3         6.0
8      75 - 79          3         6.0
9      50 - 54          3         6.0
10     25 - 29          2         4.0
11     35 - 39          2         4.0
12         N/A          1         2.0
13         80+          1         2.0


Calculate the weighted percentages using our function. We'll also map the integer values to the description values using our maps, for a more informative result.

In [ ]:
# Use the weighted_percentage function, but pass the original numeric column
weighted_percentage_result = weighted_percentage(df, 'ageg5yr', 'llcpwt')
weighted_percentage_result['value_label'] = weighted_percentage_result['ageg5yr'].map(int_to_string_map)
weighted_percentage_result = weighted_percentage_result[['value_label', 'weighted_percentage']]

print(weighted_percentage_result)


   value_label  weighted_percentage
0      18 - 24            16.482774
1      30 - 34            13.987612
2      40 - 44            12.814590
3      50 - 54            11.569552
4      45 - 49             9.932120
5      55 - 59             6.392023
6      35 - 39             6.017969
7      60 - 64             5.728505
8      25 - 29             5.037731
9      65 - 69             5.035625
10     70 - 74             3.595042
11     75 - 79             1.512207
12         N/A             1.306408
13         80+             0.587843


Merge the frequency table with the weighted percentages table

In [ ]:
# Merge the frequency data with the weighted percentages
result = pd.merge(frequency, weighted_percentage_result, on='value_label')
print(result)

   value_label  frequency  percentage  weighted_percentage
0      60 - 64          6        12.0             5.728505
1      40 - 44          5        10.0            12.814590
2      65 - 69          5        10.0             5.035625
3      45 - 49          5        10.0             9.932120
4      55 - 59          5        10.0             6.392023
5      18 - 24          5        10.0            16.482774
6      30 - 34          4         8.0            13.987612
7      70 - 74          3         6.0             3.595042
8      75 - 79          3         6.0             1.512207
9      50 - 54          3         6.0            11.569552
10     25 - 29          2         4.0             5.037731
11     35 - 39          2         4.0             6.017969
12         N/A          1         2.0             1.306408
13         80+          1         2.0             0.587843


We've now fetched data out of BigQuery and applied weights to demographic data! As you can see, the weighted percentages indicate that the sample set skewed older than the actual population.

Finally, let's take result and create a big query table out of it.

In [ ]:
job = client.load_table_from_dataframe(
    result,
    'rci-flotus.CDC.weighted_demo',
    job_config=bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE"  # Overwrite the table if it exists
    ),
)

# Wait for the job to complete
job.result()

LoadJob<project=rci-flotus, location=US, id=6c21b697-8122-4180-9cad-086c4ea57f22>